<a href="https://colab.research.google.com/github/paraery/parsingResume/blob/main/SpacyBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textfortraining8.csv')
lstTxt = df['text'].values.tolist()

In [19]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/testData.csv')
lstTest = df['text'].values.tolist()

In [1]:
import re
def get_tokens_with_entities(raw_text: str):
    # split the text by spaces only if the space does not occur between square brackets
    # we do not want to split "multi-word" entity value yet
    
    raw_tokens = re.split(r"\s(?![^\[]*\])", raw_text)

    # a regex for matching the annotation according to our notation [entity_value](entity_name)
    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags=re.I|re.M)

    tokens_with_entities = []
    lstEnt=[]
    txt = raw_text
    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token)
        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")
            
            # we prefix the name of entity differently
            # B- indicates beginning of an entity
            # I- indicates the token is not a new entity itself but rather a part of existing one
            #txt = raw_text[raw_text.find(str(raw_entity_value))-1 : ]
            txt = raw_text.replace(raw_text[int(raw_text.find(raw_entity_value))-1],'',1)
            txt = txt.replace('](PERSON)','')
            #print(txt)
            lstdata = ()
            entity_name = f"{raw_entity_name}"
            entName = createSubEntity(txt.find(raw_entity_value), txt.find(raw_entity_value) + len(raw_entity_value), entity_name)
            lstEnt.append(entName)
                #tokens_with_entities.append((raw_entity_token, entity_name))
    d={"entities": lstEnt}
            #print(lstEnt)
    lstdata=(txt,d)
    tokens_with_entities.append(lstdata)
    return lstdata
def createSubEntity(start,end,tag):
    s = (int(start),int(end),tag)
    return s

In [22]:
len(train),len(test)

(620, 87)

In [21]:
train=[]
train=train_data

In [18]:
#train_data=[]
for a in lstTxt:
    train_data.append(get_tokens_with_entities(a))

In [20]:
test=[]
for a in lstTest:
    test.append(get_tokens_with_entities(a))

In [23]:
import json
jsonStr = json.dumps(train_data)
import json
with open('drive/MyDrive/Spacy/train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

In [24]:
!pip install spacy_transformers
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [26]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [27]:
!python -m spacy init fill-config /content/drive/MyDrive/Spacy/base_config.cfg /content/drive/MyDrive/Spacy/config.cfg

2023-03-04 04:52:17.841699: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 04:52:17.841801: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 04:52:17.841820: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Spacy/config.cfg
You can now add your data and train your pipeline:
python -m spacy train co

In [28]:
def get_spacy_doc(file,data):
  nlp = spacy.blank("en")
  db = DocBin()
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']
    ents = []
    entity_indices = []
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices+list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
      except:
        continue
      if span is None:
        err_data = str([start,end])+" "+str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
      
      doc.ents = ents
      db.add(doc)
  return db

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
file = open('drive/MyDrive/Spacy/error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 87/87 [00:00<00:00, 180.06it/s]


In [30]:
!python -m spacy train /content/drive/MyDrive/Spacy/config.cfg --output ./outputSpacybert --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-03-04 04:53:31.080280: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 04:53:31.080376: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 04:53:31.080395: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
ℹ Saving to output directory: outputSpacybert
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-03-04 04:53:37,469] [

#TEST


In [31]:
nlp = spacy.load('outputSpacybert/model-best')

In [32]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textInResume.csv')
dataTest = df['text'].values.tolist()

In [37]:
i=0
for d in dataTest:
  doc = nlp(d)
  print("## doc ",i," ##")
  for ent in doc.ents:
    print(ent.text,"----",ent.label_)
  i = i+1
  print("   ")

## doc  0  ##
 certification ---- PERSON
 automation ---- PERSON
 proficient ---- PERSON
 erp ---- PERSON
supavit attagomol ---- PERSON
phoomchai audit ---- PERSON
   
## doc  1  ##
nisaratwisesbantao bachelor ---- PERSON
   
## doc  2  ##
thirawit jirarungroj ---- PERSON
   
## doc  3  ##
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
worawut       ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---

In [39]:
nlpLst = spacy.load('outputSpacybert/model-last')

In [40]:
i=0
for d in dataTest:
  doc = nlpLst(d)
  print("## doc ",i," ##")
  for ent in doc.ents:
    print(ent.text,"----",ent.label_)
  i = i+1
  print("   ")

## doc  0  ##
 certification ---- PERSON
 automation ---- PERSON
 proficient ---- PERSON
 erp ---- PERSON
supavit attagomol ---- PERSON
   
## doc  1  ##
nisaratwisesbantao bachelor ---- PERSON
   
## doc  2  ##
thirawit jirarungroj ---- PERSON
   
## doc  3  ##
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
worawut       ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 (

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


## doc  9  ##
nattacha kingbualuang ---- PERSON
   
## doc  10  ##
   
## doc  11  ##
nutratanon mahakhet ---- PERSON
   
## doc  12  ##
kitisak thossaensin ---- PERSON
   
## doc  13  ##
suebtas limsaihua ---- PERSON
   
## doc  14  ##
navara sirijarusvong ---- PERSON
   
## doc  15  ##
chayanit sripradit ---- PERSON
   
## doc  16  ##
pimnares puto ---- PERSON
   
## doc  17  ##
saran thitawiriyayos ---- PERSON
   
## doc  18  ##
pakpoom poodsud ---- PERSON
   
## doc  19  ##
kanittha setthapitayakul ---- PERSON
   
## doc  20  ##
parweenuch duangdenphatsawron ---- PERSON
   
## doc  21  ##
pawith panyasirikul ---- PERSON
   
## doc  22  ##
chotitouch supalanunt ---- PERSON
   
## doc  23  ##
aussadach masun ---- PERSON
   
## doc  24  ##
uhack kasetsart ---- PERSON
teeratach jearapaganon ---- PERSON
̄© ---- PERSON
̄ ̄£ ---- PERSON
   
## doc  25  ##
pratan srikamonpattanawut ---- PERSON
   
## doc  26  ##
natthapat phusrisom ---- PERSON
   
## doc  27  ##
chayanon juntarapartsavorn 